In [5]:
import os
import pandas as pd
import glob

from pathlib import Path

In [6]:
dir_path = Path(os.getcwd())
data_dir = str(os.path.join(dir_path.parent,'data'))
balance_files = glob.glob(data_dir + "/poker_balances*.csv")
hands_files = glob.glob(data_dir + "/poker_hands*.csv")

def get_hands_df():
    hands_dfs=[]

    for file in hands_files:
        new_df = pd.read_csv(file)
        hands_dfs.append(new_df)

    hands = pd.concat(hands_dfs)
    return hands
    
def get_balance_df():
    balance_dfs=[]
    
    for file in balance_files:
        new_df = pd.read_csv(file)
        balance_dfs.append(new_df)
        
    balances = pd.concat(balance_dfs)
    return balances


In [7]:
hands = get_hands_df()
balances = get_balance_df()

In [8]:
pd.set_option('display.max_columns', None)  
balances

,player_name,player_type,beginning_balance,balance_0,balance_1,balance_2,balance_3,balance_4,balance_5,balance_6,balance_7,balance_8,balance_9,balance_10,balance_11,balance_12,balance_13,balance_14,balance_15,balance_16,balance_17,balance_18,balance_19,balance_20,balance_21,balance_22,balance_23,balance_24,balance_25,balance_26,balance_27,balance_28,balance_29,balance_30,balance_31,balance_32,balance_33,balance_34,balance_35,balance_36,balance_37,balance_38,balance_39,balance_40,balance_41,balance_42,balance_43,balance_44,balance_45,balance_46,balance_47,balance_48,balance_49,balance_50,balance_51,balance_52,balance_53,balance_54,balance_55,balance_56,balance_57,balance_58,balance_59,balance_60,balance_61,balance_62,balance_63,balance_64,balance_65,balance_66,balance_67,balance_68,balance_69,balance_70,balance_71,balance_72,balance_73,balance_74,balance_75,balance_76,balance_77,balance_78,balance_79,balance_80,balance_81,balance_82,balance_83,balance_84,balance_85,balance_86,balance_87,balance_88,balance_89,balance_90,balance_91,balance_92,balance_93,balance_94,balance_95,balance_96,balance_97,balance_98,balance_99,balance_100,balance_101,balance_102,balance_103,balance_104,balance_105,balance_106,balance_107,balance_108,balance_109,balance_110,balance_111,balance_112,balance_113,balance_114,balance_115,balance_116,balance_117,balance_118,balance_119,balance_120,balance_121,balance_122,balance_123,balance_124,balance_125,balance_126,balance_127,balance_128,balance_129,balance_130,balance_131,balance_132,balance_133,balance_134,balance_135,balance_136,balance_137,balance_138,balance_139,balance_140,balance_141,balance_142,balance_143,balance_144,balance_145,balance_146,balance_147,balance_148,balance_149,balance_150,balance_151,balance_152,balance_153,balance_154,balance_155,balance_156,balance_157,balance_158,balance_159,balance_160,balance_161,balance_162,balance_163,balance_164,balance_165,balance_166,balance_167,balance_168,balance_169,balance_170,balance_171,balance_172,balance_173,balance_174,balance_175,balance_176,balance_177,balance_178,balance_179,balance_180,balance_181,balance_182,balance_183,balance_184,balance_185,balance_186,balance_187,balance_188,balance_189,balance_190,balance_191,balance_192,balance_193,balance_194,balance_195,balance_196,balance_197,balance_198,balance_199,balance_200,balance_201,balance_202,balance_203,balance_204,balance_205,balance_206,balance_207,balance_208,balance_209,balance_210,balance_211,balance_212,balance_213,balance_214,balance_215,balance_216,balance_217,balance_218,balance_219,balance_220,balance_221,balance_222,balance_223,balance_224,balance_225,balance_226,balance_227,balance_228,balance_229,balance_230,balance_231,balance_232,balance_233,balance_234,balance_235,balance_236,balance_237,balance_238,balance_239,balance_240,balance_241,balance_242,balance_243,balance_244,balance_245,balance_246,balance_247,balance_248,balance_249,balance_250,balance_251,balance_252,balance_253,balance_254,balance_255,balance_256,balance_257,balance_258,balance_259,balance_260,balance_261,balance_262,balance_263,balance_264,balance_265,balance_266,balance_267,balance_268,balance_269,balance_270,balance_271,balance_272,balance_273,balance_274,balance_275,balance_276,balance_277,balance_278,balance_279,balance_280,balance_281,balance_282,balance_283,balance_284,balance_285,balance_286,balance_287,balance_288,balance_289,balance_290,balance_291,balance_292,balance_293,balance_294,balance_295,balance_296,balance_297,balance_298,balance_299,balance_300,balance_301,balance_302,balance_303,balance_304,balance_305,balance_306,balance_307,balance_308,balance_309,balance_310,balance_311,balance_312,balance_313,balance_314,balance_315,balance_316,balance_317,balance_318,balance_319,balance_320,balance_321,balance_322,balance_323,balance_324,balance_325,balance_326,balance_327,balance_328,balance_329,balance_330,balance_331,balance_332,balance_333,balance_334,balance_335,balance_336,balance_337,balance_338

In [10]:
hands

,game_id,player_name,player_type,bet_number,opponents,call,current,final,pot,allowed,hand1,hand2,community1,community2,community3,community4,community5
0,1,players - standard - 1,AlwaysCallPlayer,1,5,10,0,10.0,15,True,4-hearts,Q-diamonds,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A
1,1,players - standard - 1,AlwaysCallPlayer,2,5,10,10,20.0,70,True,4-hearts,Q-diamonds,7-diamonds,A-clubs,Q-hearts,Z-N/A,Z-N/A
2,1,players - standard - 1,AlwaysCallPlayer,3,5,0,20,20.0,120,True,4-hearts,Q-diamonds,7-diamonds,A-clubs,Q-hearts,Z-N/A,Z-N/A
3,1,players - standard - 1,AlwaysCallPlayer,4,5,0,20,20.0,120,True,4-hearts,Q-diamonds,7-diamonds,A-clubs,K-hearts,Q-hearts,Z-N/A
4,1,players - standard - 1,AlwaysCallPlayer,5,5,10,20,30.0,130,True,4-hearts,Q-diamonds,7-diamonds,8-clubs,A-clubs,K-hearts,Q-hearts
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43039,2000,players - smart - 6,SmartPlayer,3,5,0,20,20.0,120,True,6-hearts,A-diamonds,10-diamonds,2-clubs,7-hearts,Z-N/A,Z-N/A
43040,2000,players - smart - 6,SmartPlayer,4,5,0,20,30.0,120,True,6-hearts,A-diamonds,10-diamonds,2-clubs,2-spades,7-hearts,Z-N/A
43041,2000,players - smart - 6,SmartPlayer,5,5,0,30,40.0,180,True,6-hearts,A-diamonds,10-diamonds,2-clubs,2-spades,7-hearts,Z-N/A
43042,2000,players - smart - 6,SmartPlayer,6,5,0,40,40.0,240,False,6-hearts,A-diamonds,10-diamonds,2-clubs,2-spades,7-hearts,Z-N/A
